In [1]:
import pyaudio
import wave
import speech_recognition as sr
import numpy as np
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
from scipy.fftpack import fft
from scipy.io.wavfile import write
import time
from tkinter.simpledialog import askstring
from matplotlib import pyplot as plt

SAMPLE_RATE = 8000  # Sample rate in Hz.
CHARACTER_DURATION = 0.04  # Duration of each character in seconds.
NUMBER_SAMPLES = int(SAMPLE_RATE * CHARACTER_DURATION)

font_name = "Helvetica"
font_size = 10  
button_font = (font_name, font_size)

# Encoding frequencies for each English character.
FREQUENCIES = {
    'a': [100, 1100, 2500], 'b': [100, 1100, 3000], 'c': [100, 1100, 3500],
    'd': [100, 1300, 2500], 'e': [100, 1300, 3000], 'f': [100, 1300, 3500],
    'g': [100, 1500, 2500], 'h': [100, 1500, 3000], 'i': [100, 1500, 3500],
    'j': [300, 1100, 2500], 'k': [300, 1100, 3000], 'l': [300, 1100, 3500],
    'm': [300, 1300, 2500], 'n': [300, 1300, 3000], 'o': [300, 1300, 3500],
    'p': [300, 1500, 2500], 'q': [300, 1500, 3000], 'r': [300, 1500, 3500],
    's': [500, 1100, 2500], 't': [500, 1100, 3000], 'u': [500, 1100, 3500],
    'v': [500, 1300, 2500], 'w': [500, 1300, 3000], 'x': [500, 1300, 3500],
    'y': [500, 1500, 2500], 'z': [500, 1500, 3000], ' ': [500, 1500, 3500]
}

def generate_character_signal(frequencies):
    character_signal = [
        np.cos(frequencies[0] * 2 * np.pi * n / SAMPLE_RATE) +
        np.cos(frequencies[1] * 2 * np.pi * n / SAMPLE_RATE) +
        np.cos(frequencies[2] * 2 * np.pi * n / SAMPLE_RATE)
        for n in range(NUMBER_SAMPLES)
    ]
    return character_signal

# Function to encode a string into a signal.
def encode_string_to_signal(input_string):
    signal = []
    for char in input_string:
        if char in FREQUENCIES: # if the character is in the dictionary.
            signal = np.concatenate((signal, generate_character_signal(FREQUENCIES[char])), axis=None)
        else:
            # If the character is not in the dictionary
            # add a some frequencies to the signal not in the dictionary.
            signal = np.concatenate((signal, generate_character_signal([600, 1600, 2600])), axis=None)
    return signal

# Function to get input string from the GUI text entry.
def get_input_string():
    input_text = text_entry.get("1.0", tk.END).strip().lower()
    if not input_text:
        messagebox.showerror("Error", "Please enter some text to encode.", font=(font_name, font_size))
        return
    return input_text

# Function to encode the input string and return the generated signal.
def encode():
    input_text = get_input_string()
    encoded_signal = encode_string_to_signal(input_text)
    return encoded_signal

def save_generated_signal():
    encoded_signal = encode()
    file_path = askstring('Name', 'Enter a name of file to save the generated signal:')
    if file_path and file_path[-4:] != ".wav":
        file_path += ".wav"
    if file_path:
        normalized_signal = np.int16((encoded_signal / encoded_signal.max()) * 32767)
        write(file_path, SAMPLE_RATE, normalized_signal)
        messagebox.showinfo("Success", f"File saved as {file_path}.")

# Function to play the generated signal and plot its time and frequency domains.
def play_generated_signal():
    # Generate the encoded signal.
    encoded_signal = encode()

    # Play the sound.
    sd.play(encoded_signal, SAMPLE_RATE)

    # Wait until sound has finished playing.
    time.sleep(1)

    # Plot the time and frequency domains of the signal.
    plot_signal(encoded_signal)

# Function to plot the time and frequency domains of a signal.
def plot_signal(signal):
    # Plotting
    plt.figure(figsize=(10, 6))

    # Time domain plot.
    plt.subplot(2, 1, 1)  # 2 rows, 1 column, first plot.
    time = np.linspace(0, len(signal) / SAMPLE_RATE, num=len(signal))

    plt.plot(time, signal)
    plt.title(f"Time Domain: Encoded Signal", font=font_name)
    plt.xlabel("Time (sec)", font=font_name)
    plt.ylabel("Amplitude", font=font_name)
    plt.grid(True)

    plt.subplot(2, 1, 2)
    N = len(signal)
    freq = np.fft.fftfreq(N, 1 / SAMPLE_RATE)
    freq = freq[:N // 2]
    magnitude = np.abs(fft(signal))[:N // 2]
    plt.plot(freq, magnitude)  # Plot only the positive frequencies.
    plt.title("Frequency Domain: Spectrum", font=font_name)
    plt.xlabel("Frequency (Hz)", font=font_name)
    plt.ylabel("Magnitude", font=font_name)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

def record_audio(duration=5, filename="recorded.wav"):
    FORMAT = pyaudio.paInt16  # Format of sampling
    CHANNELS = 1  # Number of audio channels
    RATE = 44100  # Sampling rate
    CHUNK = 1024  # Chunk size for buffer
    RECORD_SECONDS = duration  # Duration of recording
    WAVE_OUTPUT_FILENAME = filename  # Output filename

    audio = pyaudio.PyAudio()  # Create pyaudio instantiation

    # Create pyaudio stream
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("Recording...")

    frames = []

    # Store data in chunks for RECORD_SECONDS
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def audio_to_text(filename="recorded.wav"):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print("Transcription: " + text)
            return text.lower()
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

# Function to record audio, convert it to text, and encode it.
def record_and_encode():
    record_audio()
    recorded_text = audio_to_text()
    if recorded_text:
        text_entry.delete('1.0', tk.END)
        text_entry.insert(tk.END, recorded_text)
        encoded_signal = encode_string_to_signal(recorded_text)
        return encoded_signal
    else:
        messagebox.showerror("Error", "Failed to transcribe the recorded audio.")

if __name__ == "__main__":
    # Create the main window.
    root = tk.Tk()
    root.title("Voice-Frequency Encoder")
    root.geometry("800x400")
    root.configure(bg="#f0f0f0")

    # Title label.
    title_label = tk.Label(root, text="Voice-Frequency Encoder", font=("Helvetica", 16, "bold"), bg="#f0f0f0")
    title_label.pack(pady=10)

    # Create a frame for text entry.
    text_frame = tk.Frame(root, bg="#f0f0f0", padx=10, pady=10, highlightbackground="gray", highlightthickness=1)
    text_frame.pack(pady=10)

    # Create a text entry widget.
    text_entry = tk.Text(text_frame, height=10, width=50, font=(font_name, 12), bg="#ffffff", bd=0)
    text_entry.pack()

    button_bg_color = "lightblue"

    # Create a frame for buttons.
    button_frame = tk.Frame(root, bg="#f0f0f0")
    button_frame.pack(pady=10)

    # Create a button to trigger the recording and encoding.
    record_button = tk.Button(button_frame, text="Record Audio and Encode", command=record_and_encode,
                              font=(font_name, font_size), bg=button_bg_color, padx=20, pady=10)
    record_button.grid(row=0, column=0, padx=10, pady=5)

    # Create a button to trigger the encoding and saving.
    save_button = tk.Button(button_frame, text="Save the generated signal as (.wav)", command=save_generated_signal,
                            font=(font_name, font_size), bg=button_bg_color, padx=20, pady=10)
    save_button.grid(row=0, column=1, padx=10, pady=5)

    # Create a button to trigger the play sound.
    play_button = tk.Button(button_frame, text="Play the generated signal", command=play_generated_signal,
                            font=(font_name, font_size), bg=button_bg_color, padx=20, pady=10)
    play_button.grid(row=0, column=2, padx=10, pady=5)

    credit_label = tk.Label(root, text="Project created by Faizer & Imran & Rifat", font=("Arial", 12), fg="gray", bg="#f0f0f0")
    credit_label.pack(pady=10)

    root.mainloop()
